In [1]:
! pip install datasets transformers nltk evaluate sacrebleu ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beha

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab_Notebooks/github/NL2TL

/content/drive/MyDrive/Colab_Notebooks/github/NL2TL


In [4]:
import nltk
nltk.download('punkt_tab')
import transformers

from IPython.core import error
import json
from fnmatch import fnmatchcase as match
import random
import os
import pandas as pd
import datasets
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
import numpy as np
from argparse import ArgumentParser
from typing import Dict, List, Union
import ast
import csv
import ipdb
import ast
import evaluate

## params
int_seed = 1203
dataset_name = "LTL_koreauniv"
init_weight = "with_pre-train"
data_size = "0.1-0.9"
model_checkpoint = "t5-base"
print(model_checkpoint)
print('\n')

home_path = "/content/drive/MyDrive/Colab_Notebooks/github/NL2TL"
data_dir = "dataset"
data_filename = "command_LTL_dataset_v01.csv"
data_path = os.path.join(home_path, data_dir, data_filename)

symbol_to_word_map = {
    "F(": "finally(",
    "G(": "globally(",
    "U(": "until(",
    "&": "and",
    "¬": "not",
    "∧": "and",
    "∨": "or",
    "→": "imply",
}

# read original csv
with open(data_path, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    data = []
    for row in reader:
        data.append([ast.literal_eval(row["generated_command"]),
                        ast.literal_eval(row["generated_LTL"])])

# realign dataset format and write to csv
new_data_filename = data_filename[:-4] + "_realign.csv"
f = open(os.path.join(home_path, data_dir, new_data_filename), 'w')
csv_writer = csv.writer(f)
csv_writer.writerow(["id", "ltl", "sentence"])
i = 1
for item in data:
    sents, ltls = item[0], item[1]
    for s in sents:
        for l in ltls:
            # convert symbol to word
            for k, v in symbol_to_word_map.items():
                l = l.replace(k, v)
            csv_writer.writerow([i, l, s])
            i += 1
f.close()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


t5-base




In [5]:
from transformers import AutoTokenizer

# path
input_model_dir = os.path.join(home_path, "model", "t5-base-epch20-infix-word-04-21", "checkpoint-62500")
output_model_dir = os.path.join(home_path, "model", "t5-base-transfer-learning/")
if not os.path.exists(output_model_dir):
  os.mkdir(output_model_dir)

# tokenizer and params
tokenizer = AutoTokenizer.from_pretrained(input_model_dir)
max_input_length = 1024
max_target_length = 128
prefix = "Transform the following sentence into Signal Temporal logic: "

# preprocess data
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["sentence"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["ltl"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    model_inputs["sentence"] = examples["sentence"]
    model_inputs["ltl"] = examples["ltl"]
    model_inputs["id"] = examples["id"]
    return model_inputs

# define compute metrics
bleu_metric = evaluate.load("bleu", force_prefix=True)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # print(predictions)
    # print(labels)
    # Replace -100 in the labels as we can't decode them.
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    count = 0

    # top-1 accuracy
    for i in range(len(decoded_preds)):
        pred = nltk.sent_tokenize(decoded_preds[i].strip())
        label = nltk.sent_tokenize(decoded_labels[i].strip())
        if pred == label:
            count += 1

    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {'top-1 accuracy': round(count / len(decoded_preds), 6),
            'bleu score': bleu['bleu'],
            'bleu precisions': bleu['precisions']}

# correct parenthesis
def correct_parenthe(input_str):
  count = 0
  original_list = input_str.split(' ')
  for index, item in enumerate(original_list):
    if len(item) >2:
      if item[-1] == '.':
        original_list[index] = original_list[index][:-1]
    if item == '(':
      count += 1
    elif item == ')':
      count -= 1
  if count >0:
    for i in range(count):
      original_list.append(')')
  if count <0:
    for i in range(-count):
      original_list.pop(-1)
  return ' '.join(original_list)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The below is the training code block. So you would nee to run the below for transfer-learning fit to the specific dataset.

In [8]:
# transfer learning
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# load realigned dataset
dataset = load_dataset('csv', data_files=os.path.join(home_path, data_dir, new_data_filename))
train_dataset, test_dataset = dataset['train'].train_test_split(test_size=0.1, shuffle=True, seed=int_seed).values()
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# load pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(input_model_dir)
batch_size = 16
output_model_name = model_checkpoint.split("/")[-1]+'-'+dataset_name+"-epoch20-trainpoint"+str(i+1)
output_model_dir = output_model_dir+output_model_name

# set trainer params
args = Seq2SeqTrainingArguments(
    output_model_dir,
    output_model_name,
    evaluation_strategy = "steps",
    eval_steps=100,
    logging_steps=100,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    seed=int_seed,
    save_total_limit=1,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=False,
    #push_to_hub=True,
    #report_to="tensorboard",
    #load_best_model_at_end=True,
    #save_strategy = "no"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset ,
    eval_dataset=tokenized_test_dataset ,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# run train
trainer.train()
trainer.save_model()

Map:   0%|          | 0/1872 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-4a3fd3381297>:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss,Top-1 accuracy,Bleu score,Bleu precisions
100,1.320600,0.324598,0.033654,0.230745,"[0.857055589492975, 0.6627011896431071, 0.4176904176904177, 0.26159921026653504]"
200,0.397600,0.226343,0.028846,0.280199,"[0.8923529411764706, 0.7158176943699732, 0.4883177570093458, 0.33271375464684017]"
300,0.301500,0.188934,0.038462,0.307298,"[0.8934097421203439, 0.7267404033832141, 0.5139202407825433, 0.3773416592328278]"
400,0.260000,0.167626,0.028846,0.309110,"[0.8825503355704698, 0.7037974683544304, 0.49489795918367346, 0.35738831615120276]"
500,0.234800,0.157109,0.052885,0.331374,"[0.8946200776483638, 0.7379310344827587, 0.5212689257390051, 0.3994910941475827]"
600,0.217200,0.144740,0.038462,0.331397,"[0.8737127371273713, 0.722052535125229, 0.5052484254723583, 0.3726453726453726]"
700,0.204900,0.136386,0.043269,0.325964,"[0.876226826608506, 0.7214022140221402, 0.5, 0.3652892561983471]"
800,0.192000,0.131472,0.057692,0.354200,"[0.8796592119275826, 0.7371257485029941, 0.5294117647058824, 0.4043062200956938]"
900,0.183100,0.129062,0.052885,0.355570,"[0.8804004214963119, 0.7390532544378698, 0.5182186234817814, 0.391679748822606]"
1000,0.177800,0.125676,0.052885,0.351565,"[0.8830687830687831, 0.7300832342449465, 0.5169606512890095, 0.3886255924170616]"


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "[0.857055589492975, 0.6627011896431071, 0.4176904176904177, 0.26159921026653504]" of type <class 'list'> for key "eval/bleu precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8923529411764706, 0.7158176943699732, 0.4883177570093458, 0.33271375464684017]" of type <class 'list'> for key "eval/bleu precisions" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.8934097421203439, 0.7267404033832141, 0.5139202407825433, 0.3773416592328278]" of type <class 'list'>

The below code block is for testing using the above transfer-learned model or already pre-trained model. If you want to only test without training, then run from the first code block but skip the just previous training code block and run the below code block.

In [9]:
# Test with a transfer learning model
import torch
import evaluate
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
show_diff_parenthe_results = False
output_model_dir = os.path.join(home_path, "model", "t5-base-transfer-learning", f"t5-base-{dataset_name}-epoch20-trainpoint2082/checkpoint-2340")
#output_model_dir = os.path.join(home_path, "model", "t5-base-epch20-infix-word-04-21", "checkpoint-62500")

model = AutoModelForSeq2SeqLM.from_pretrained(output_model_dir).to(device)
acc_top1 = 0.
acc_bleu_scores = 0.
acc_bleu_precisions = [0., 0., 0., 0.]

metric = evaluate.load("accuracy")
dataset = load_dataset('csv', data_files=os.path.join(home_path, data_dir, new_data_filename))
train_dataset, test_dataset = dataset['train'].train_test_split(test_size=0.1).values()
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
prefix = "Transform the following sentence into Signal Temporal logic: "

with open(output_model_dir +'/result.txt', 'w') as f_result:
    for i in range(len(tokenized_test_dataset)):
        input = [prefix + tokenized_test_dataset[i]['sentence']]
        f_result.write("input: " + input[0] + '\n')

        input = tokenizer(input, max_length=max_input_length, truncation=True, return_tensors="pt").to(device)
        output = model.generate(**input, num_beams=8, do_sample=True, min_length=10, max_length=64)
        decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
        label = tokenized_test_dataset[i]['ltl']
        predicted_output = correct_parenthe(decoded_output.strip())
        f_result.write("label: " + label + '\n')
        f_result.write("pred: " + predicted_output + '\n')
        f_result.write('\n')

        if predicted_output == label:
            acc_top1 += 1

        bleu = bleu_metric.compute(predictions=[predicted_output], references=[label])
        acc_bleu_scores += bleu['bleu']
        acc_bleu_precisions = [acc_bleu_precisions[n] + j for n, j in enumerate(bleu['precisions'])]

    test_data_num = i + 1
    acc_top1 /= test_data_num
    acc_bleu_scores /= test_data_num
    acc_bleu_precisions = [j / test_data_num for j in acc_bleu_precisions]

    print(f"The test data number = {test_data_num}")
    print(f"Top-1 accuracy = {acc_top1}")
    print(f"Bleu score = {acc_bleu_scores}")
    print(f"Bleu precision = {acc_bleu_precisions}")
    f_result.write('\n')
    f_result.write(f"The test data number = {test_data_num}\n")
    f_result.write(f"Top-1 accuracy = {acc_top1}\n")
    f_result.write(f"Bleu score = {acc_bleu_scores}\n")
    f_result.write(f"Bleu precision = {acc_bleu_precisions}\n")

f_result.close()

The test data number = 208
Top-1 accuracy = 0.09615384615384616
Bleu score = 0.5354062188772656
Bleu precision = [0.8618050151685388, 0.7313585529576325, 0.5328485424468097, 0.39822951716585675]
